In [ ]:
import os

os.chdir('../')

## Some analysis

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data_train_path = "Dataset/data_train.csv"

URM_all_dataframe = pd.read_csv(data_train_path)
print(URM_all_dataframe.head(10))

   user_id  item_id  data
0        0        0   1.0
1        0        2   1.0
2        0      120   1.0
3        0      128   1.0
4        0      211   1.0
5        0      232   1.0
6        0      282   1.0
7        0      453   1.0
8        0      458   1.0
9        0      491   1.0


In [ ]:
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout



URM_all = sps.coo_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_all = URM_all.tocsr()

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [ ]:
from Recommenders.SLIM.SLIMElasticNetRecommender_Peppe import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender

recommender_slimNet = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train_validation)
recommender_slimNet.fit(topK=142, alpha = 9.521340863590419e-05, l1_ratio = 0.4033590645217344)

recommender_easeR = EASE_R_Recommender(URM_train_validation)
recommender_easeR.fit(l2_norm=33.20)

# Optuna Study

In [ ]:
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

def objective_function(trial):

    alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


    recommender = ItemKNNCustomSimilarityRecommender(URM_train_validation)

    new_similarity = recommender_slimNet.W_sparse * alpha + recommender_rp3.W_sparse * (1-alpha)

    recommender.fit(new_similarity)

    result_dict, _ = evaluator_test.evaluateRecommender(recommender)

    MAP = result_dict.loc[10]["MAP"]
    
    return MAP

In [ ]:
class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/Hybrid/hybrid_slim_EASER_MM.csv", index = False)

In [ ]:
import optuna

optuna_study = optuna.create_study(study_name="hybrid_slim_ep3", direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 100
                      )